In [1]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 14.4 MB/s 
     |████████████████████████████████| 596 kB 65.5 MB/s 
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 6.6 MB 59.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
drive_path = '/content/drive/MyDrive/Colab Notebooks/구름 NLP 과정 project3/'

##데이터 전처리

In [4]:
import unicodedata

In [5]:
data_dev = []
with open(drive_path + 'jit/je.dev', 'r', encoding = "UTF-8") as fd:
    for line in fd.readlines():
        data_dev.append(line[:-1])

data_test = []
with open(drive_path + 'jit/je.test', 'r', encoding = "UTF-8") as fd:
    for line in fd.readlines():
        data_test.append(line[:-1])

data_train = []
with open(drive_path + 'jit/je.train', 'r', encoding = "UTF-8") as fd:
    for line in fd.readlines():
        data_train.append(line[:-1])

In [6]:
dot = data_dev[0][2]

In [7]:
two_dot = data_dev[21][39]

In [8]:
data = data_dev + data_test + data_train
changed_data = []
dot_num = 0
two_dot_num = 0

for line in data:
    dot_num += line.count(dot)
    two_dot_num += line.count(two_dot)
    newline = line.replace(dot,'ㅗ')
    changed_data.append(unicodedata.normalize('NFKC',newline.replace(two_dot,'ㅛ')))
print(changed_data[0], dot_num, two_dot_num, len(changed_data))

경호민 이제 그것에 고치 씨에 버물려지민 걸름에 다 볿을 때 볿앙 요만씩요만씩 다 뿌려노민이 , 그것에 붙엉 씨가 나오민 호꼼 걸어라게 . 그냥 가는 건 쉡씨 , 그냥 걸름 아니 헹 그냥 씨삐엉 가는 건 그것엔 걸름이 들어가난 호 잘 뒈어라게 밀랑밀랑하게 . 75733 1001 170356


In [9]:
print(changed_data[0][1])
if '호' == changed_data[0][1]:
    print('일치')
else:
    print('불일치')

호
일치


In [10]:
print(len(data_dev), len(data_test), len(data_train))

5000 5000 160356


In [11]:
with open(drive_path + 'jit/je_trans.dev', 'w') as fd:
    for line in changed_data[:5000]:
        fd.write(line+'\n')

with open(drive_path + 'jit/je_trans.test', 'w') as fd:
    for line in changed_data[5000:10000]:
        fd.write(line+'\n')

with open(drive_path + 'jit/je_trans.train', 'w') as fd:
    for line in changed_data[10000:]:
        fd.write(line+'\n')

##토크나이저

In [16]:
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import json

from transformers import (
    EncoderDecoderModel,
    PreTrainedTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Trainer,
    BertConfig,
    DistilBertConfig,
    EncoderDecoderConfig
)

import torch
from torch.utils.data import Dataset, DataLoader

In [17]:
limit_alphabet = 4000
vocab_size = 32000

tokenizer = BertWordPieceTokenizer(
    vocab=None,
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # Must be False if cased model
    lowercase=False,
    wordpieces_prefix="##"
)
files = [drive_path + 'jit/je.dev', drive_path + 'jit/je.train', drive_path + 'jit/je.test']
tokenizer.train(
    files=files,
    limit_alphabet=limit_alphabet,
    vocab_size=vocab_size
)

tokenizer.save(drive_path + "dump/ch-{}-je-{}-pretty".format(limit_alphabet, vocab_size),True)

files = [drive_path + 'jit/je_trans.dev', drive_path + 'jit/je_trans.train', drive_path + 'jit/je_trans.test']
tokenizer.train(
    files=files,
    limit_alphabet=limit_alphabet,
    vocab_size=vocab_size
)
tokenizer.save(drive_path + "dump/ch-{}-je_trans-{}-pretty".format(limit_alphabet, vocab_size),True)

In [18]:
vocab_path = [drive_path + "dump/ch-4000-je-32000-pretty", drive_path + "dump/ch-4000-je_trans-32000-pretty"]
vocab_file = [drive_path + 'dump/wpm-vocab-je.txt', drive_path + 'dump/wpm-vocab-je_trans.txt']

for i in range(len(vocab_path)):
    f = open(vocab_file[i],'w',encoding='utf-8')
    with open(vocab_path[i]) as json_file:
        json_data = json.load(json_file)
        for item in json_data["model"]["vocab"].keys():
            f.write(item+'\n')

        f.close()

In [19]:
src_tokenizer = BertTokenizer(drive_path + 'dump/wpm-vocab-je_trans.txt', do_lower_case=False)
trg_tokenizer = BertTokenizer(drive_path + 'dump/wpm-vocab-je.txt', do_lower_case=False)

In [20]:
def tokenized_data(src_tokenizer, trg_tokenizer, src_file_path, trg_file_path):
    src_data = []
    trg_data = []
    with open(src_file_path, 'r') as fd:
        for line in fd.readlines():
            src_data.append(line[:-1])
    with open(trg_file_path, 'r') as fd:
        for line in fd.readlines():
            trg_data.append(line[:-1])

    embeddings = []
    for i in range(len(src_data)):
        embeddings.append((src_tokenizer(src_data[i], truncation=True, max_length=50, return_token_type_ids=False, padding='max_length'), \
                          trg_tokenizer(trg_data[i], truncation=True, max_length=50, return_token_type_ids=False, padding='max_length')))

    return embeddings

In [21]:
train_tokenized_data = tokenized_data(src_tokenizer, trg_tokenizer, drive_path + 'jit/je_trans.train', drive_path + 'jit/je.train')
dev_tokenized_data = tokenized_data(src_tokenizer, trg_tokenizer, drive_path + 'jit/je_trans.dev', drive_path + 'jit/je.dev')
test_tokenized_data = tokenized_data(src_tokenizer, trg_tokenizer, drive_path + 'jit/je_trans.test', drive_path + 'jit/je.test')

In [22]:
train_tokenized_data[0]

({'input_ids': [2, 8688, 5919, 65, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
 {'input_ids': [2, 8833, 6021, 95, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]})

In [23]:
class DatasetRetriever(Dataset):
    def __init__(self, features):
        super(DatasetRetriever, self).__init__()
        self.features = features
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):   
        feature = self.features[index]
        return {
            'input_ids':torch.tensor(feature[0]['input_ids'] ,dtype=torch.long),
            #'attention_mask':torch.tensor(feature[0]['attention_mask'], dtype=torch.long),
            'labels':torch.tensor(feature[1]['input_ids'] ,dtype=torch.long)
            #'decoder_input_ids':torch.tensor(feature[1]['input_ids'] ,dtype=torch.long),
            #'decoder_attention_mask':torch.tensor(feature[1]['attention_mask'], dtype=torch.long)
        }

In [24]:
train_dataset = DatasetRetriever(train_tokenized_data)
dev_dataset = DatasetRetriever(dev_tokenized_data)
test_dataset = DatasetRetriever(test_tokenized_data)

In [25]:
train_dataset[3]

{'input_ids': tensor([   2, 1354,    6, 4536,    7,    3,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0]),
 'labels': tensor([   2, 1369,    6, 4626,    7,    3,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0])}

In [26]:
len(train_dataset)

160356

##모델

In [27]:
config = BertConfig()
config.vocab_size = vocab_size
config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

In [28]:
config_encoder = config
config_decoder = config

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

# Initializing a Bert2Bert model from the bert-base-uncased style configurations
model = EncoderDecoderModel(config=config)

# Accessing the model configuration
config_encoder = model.config.encoder
config_decoder = model.config.decoder

# set decoder config to causal lm
config_decoder.is_decoder = True
config_decoder.add_cross_attention = True

# Saving the model, including its configuration
model.save_pretrained("my-model")

# loading model and config from pretrained folder
encoder_decoder_config = EncoderDecoderConfig.from_pretrained("my-model")
model = EncoderDecoderModel.from_pretrained("my-model", config=encoder_decoder_config)

In [29]:
model.config.decoder_start_token_id = trg_tokenizer.cls_token_id
model.config.pad_token_id = trg_tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

In [30]:
collator = DataCollatorForSeq2Seq(src_tokenizer, model)

arguments = Seq2SeqTrainingArguments(
    output_dir = 'dump3',
    do_train = True, 
    do_eval = True,
    evaluation_strategy= 'epoch',
    save_strategy='epoch',
    num_train_epochs = 10,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    warmup_ratio = 0.1,
    gradient_accumulation_steps = 1,
    save_total_limit=5,
    dataloader_num_workers= 1,
    #fp16 = True,
    load_best_model_at_end = True
)

trainer = Trainer(
    model, 
    arguments,
    data_collator = collator,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained(drive_path + "dump3/models")